In [70]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.optimize import nnls
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv('./Non-square-sp-smdm-memory.csv',encoding='euc-kr')
df

,lr,lc,rc,ld,rd,lnnz,rnnz,저장전메모리(작성),저장후메모리(작성),"좌측, 우측 희소행렬 저장",todense후메모리(작성),우측 희소행렬 toDense,곱셈후메모리(작성),곱셈,총메모리합
0,11636,20031,11259,0.016000,0.022117,3729601,4988188,10.70,15.6,4.90,15.7,0.1,15.8,0.1,5.10
1,472,155935,3626,0.042674,0.005037,3141609,2848497,9.98,12.4,2.42,15.2,2.8,15.3,0.1,5.32
2,34369,19457,9425,0.002993,0.017991,2001484,3299434,10.80,14.3,3.50,14.4,0.1,16.2,1.8,5.40
3,17922,160686,714,0.001152,0.038738,3317532,4447011,10.30,15.0,4.70,15.0,0.0,16.0,1.0,5.70
4,9581,16824,19275,0.047721,0.008130,7693122,2636477,10.50,17.7,7.20,17.8,0.1,17.9,0.1,7.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5579,138770,12555,0.020277,0.030960,15700344,53942019,10.10,56.3,46.20,69.3,13.0,69.4,0.1,59.30
96,26061,51521,37087,0.037411,0.026095,50231125,49861395,10.50,77.6,67.10,77.6,0.0,77.6,0.0,67.10
97,35625,79414,11287,0.016000,0.054465,45267635,48824425,10.80,79.1,68.30,79.1,0.0,79.2,0.1,68.40
98,25865,65104,18847,0.035231,0.045996,59328892,56438830,10.60,77.3,66.70,86.5,9.2,90.2,3.7,79.60


In [71]:
from sklearn.metrics import r2_score

def mean_absolute_percentage_error(y_test, y_pred):
    y_test, y_pred = np.array(y_test), np.array(y_pred)
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

In [72]:
feature = pd.DataFrame(df['lnnz'], columns=['lnnz'])
feature['lc*rc'] = pd.DataFrame(df['lc'] * df['rc'],columns=['lc*rc'])
feature['lr*rc'] = pd.DataFrame(df['lr'] * df['rc'],columns=['lr*rc'])
target = df['총메모리합']
feature

,lnnz,lc*rc,lr*rc
0,3729601,225529029,131009724
1,3141609,565420310,1711472
2,2001484,183382225,323927825
3,3317532,114729804,12796308
4,7693122,324282600,184673775
...,...,...,...
95,15700344,1742257350,70044345
96,50231125,1910759327,966524307
97,45267635,896345818,402099375
98,59328892,1227015088,487477655


In [93]:
epoch = 100
mape = []
r2 = []

for i in range(epoch):
    
    # 훈련 데이터와 테스트 데이터 9:1 분리
    X_train, X_test, Y_train, Y_test = train_test_split(feature, target, test_size = 0.1)

    # 변형 객체 생성
    minmax_scaler = MinMaxScaler()

    # 훈련데이터의 모수 분포 저장
    minmax_scaler.fit(X_train)

    # 훈련 데이터 스케일링
    X_train = minmax_scaler.transform(X_train)

    # 테스트 데이터의 스케일링
    X_test = minmax_scaler.transform(X_test)
    
    # NNLS(Non-Negative Least Squares)모델 정의 및 X_train 학습
    # (모든 가중치가 0 또는 양수값을 가진다.)
    np.set_printoptions(precision=8, suppress=True)
    nnls_coef, nnls_intercept = nnls(X_train, Y_train, maxiter=1000)

    # NNLS를 통해 나온 각 계수 추출
    C1=np.array(pd.DataFrame(nnls_coef).iloc[0])
    C2=np.array(pd.DataFrame(nnls_coef).iloc[1])
    C3=np.array(pd.DataFrame(nnls_coef).iloc[2])

    X_train = pd.DataFrame(X_train, columns=['lnnz','lc*rc','lr*rc'])
    X_test = pd.DataFrame(X_test, columns=['lnnz','lc*rc','lr*rc'])
    
    # X_test 예측
    nnls_Y_pred = C1*X_test['lnnz'] + C2*X_test['lc*rc'] + C3*X_test['lr*rc']
    
    # 모델의 weight(가중치)와 bias(절편) 출력
    #print("nnls_coef : ",nnls_coef)
    #print("\nnnls_intercept : ",nnls_intercept)

#     # nnls metric
#     print("\nMAPE : ", mean_absolute_percentage_error(Y_test,nnls_Y_pred))
#     print("R^2 : ", r2_score(Y_test, nnls_Y_pred))

    #print(nnls_coef)

    mape.append(mean_absolute_percentage_error(Y_test,nnls_Y_pred))
    r2.append(r2_score(Y_test, nnls_Y_pred))

print("MAPE : ", sum(mape)/len(mape) )
print("R^2 : ", sum(r2)/len(r2) )

MAPE :  23.749813992142954
R^2 :  0.5797748783296288


In [89]:
epoch = 100
mape = []
r2 = []

for i in range(epoch):
    
    # 훈련 데이터와 테스트 데이터 9:1 분리
    X_train, X_test, Y_train, Y_test = train_test_split(feature, target, test_size = 0.1)

    # NNLS(Non-Negative Least Squares)모델 정의 및 X_train 학습
    # (모든 가중치가 0 또는 양수값을 가진다.)
    np.set_printoptions(precision=8, suppress=True)
    nnls_coef, nnls_intercept = nnls(X_train.to_numpy(), Y_train.to_numpy(), maxiter=1000)

    # NNLS를 통해 나온 각 계수 추출
    C1=np.array(pd.DataFrame(nnls_coef).iloc[0])
    C2=np.array(pd.DataFrame(nnls_coef).iloc[1])
    C3=np.array(pd.DataFrame(nnls_coef).iloc[2])

    # X_test 예측
    nnls_Y_pred = C1*X_test['lnnz'] + C2*X_test['lc*rc'] + C3*X_test['lr*rc']

    # 모델의 weight(가중치)와 bias(절편) 출력
    #print("nnls_coef : ",nnls_coef)
    #print("\nnnls_intercept : ",nnls_intercept)

#     # nnls metric
#     print("\nMAPE : ", mean_absolute_percentage_error(Y_test,nnls_Y_pred))
#     print("R^2 : ", r2_score(Y_test, nnls_Y_pred))

#     print(nnls_coef)

    mape.append(mean_absolute_percentage_error(Y_test,nnls_Y_pred))
    r2.append(r2_score(Y_test, nnls_Y_pred))

print("MAPE : ", sum(mape)/len(mape) )
print("R^2 : ", sum(r2)/len(r2) )

[0.00000064 0.00000001 0.        ]
[0.00000066 0.00000001 0.        ]
[0.00000066 0.00000001 0.        ]
[0.00000068 0.00000001 0.        ]
[0.00000067 0.00000001 0.        ]
[0.00000069 0.00000001 0.        ]
[0.0000007  0.00000001 0.        ]
[0.00000064 0.00000001 0.        ]
[0.00000068 0.00000001 0.        ]
[0.00000067 0.00000001 0.        ]
[0.00000071 0.00000001 0.        ]
[0.00000065 0.00000001 0.        ]
[0.00000064 0.00000001 0.        ]
[0.00000068 0.00000001 0.00000001]
[0.00000064 0.00000001 0.        ]
[0.00000066 0.00000001 0.        ]
[0.00000064 0.00000001 0.        ]
[0.00000068 0.00000001 0.        ]
[0.00000067 0.00000001 0.        ]
[0.00000065 0.00000001 0.        ]
[0.00000067 0.00000001 0.        ]
[0.00000068 0.00000001 0.        ]
[0.00000068 0.00000001 0.        ]
[0.0000007  0.00000001 0.        ]
[0.00000066 0.00000001 0.        ]
[0.00000066 0.00000001 0.        ]
[0.00000064 0.00000001 0.        ]
[0.00000067 0.00000001 0.        ]
[0.00000068 0.000000

In [60]:
# feature, target 값 정의
feature = df[['lr','lc','rc','ld','rd','lnnz','rnnz']]
feature = feature.astype({'lr': 'int','lc': 'int','rc': 'int','lnnz': 'int','rnnz': 'int'})
target = df['총메모리합']

epoch = 100
mape = []
r2 = []

for i in range(epoch):
    
    # 훈련 데이터와 테스트 데이터 분리
    X_train, X_test, Y_train, Y_test = train_test_split(feature, target, test_size = 0.1)

    # NNLS(Non-Negative Least Squares)회귀모델 정의 및 훈련데이터로 학습
    # (모든 가중치가 0 또는 양수값을 가진다.)
    np.set_printoptions(precision=7, suppress=True)
    nnls_coef, nnls_intercept =nnls(X_train.to_numpy(), Y_train.to_numpy(), maxiter=1000)

    # NNLS를 통해 나온 각 weight 추출
    C1=np.array(pd.DataFrame(nnls_coef).iloc[0])
    C2=np.array(pd.DataFrame(nnls_coef).iloc[1])
    C3=np.array(pd.DataFrame(nnls_coef).iloc[2])
    C4=np.array(pd.DataFrame(nnls_coef).iloc[3])
    C5=np.array(pd.DataFrame(nnls_coef).iloc[4])
    C6=np.array(pd.DataFrame(nnls_coef).iloc[5])
    C7=np.array(pd.DataFrame(nnls_coef).iloc[6])

    # 테스트 데이터 예측값 도출
    nnls_Y_pred = C1*X_test['lr']+C2*X_test['lc']+C3*X_test['rc']+C4*X_test['ld']+C5*X_test['rd']+C6*X_test['lnnz']+C7*X_test['rnnz']

    # 모델의 weight(가중치)와 bias(절편) 출력
    #print("nnls_coef : ",nnls_coef)
    #print("\nnnls_intercept : ",nnls_intercept)

    # nnls metric
    #print("\nMAPE : ", mean_absolute_percentage_error(Y_test,nnls_Y_pred))
    #print("R^2 : ", r2_score(Y_test, nnls_Y_pred))

    print(str(nnls_coef) + "\n")    
    
    mape.append(mean_absolute_percentage_error(Y_test,nnls_Y_pred))
    r2.append(r2_score(Y_test, nnls_Y_pred))

print("MAPE : ", sum(mape)/len(mape) )
print("R^2 : ", sum(r2)/len(r2) )

[ 0.0000244  0.0000109  0.0000764  0.        29.9441178  0.0000007
  0.0000006]

[ 0.0000289  0.0000061  0.0000687  8.9730252 18.542193   0.0000006
  0.0000007]

[ 0.0000245  0.0000077  0.0000831  5.078137  19.9361471  0.0000007
  0.0000006]

[ 0.0000258  0.0000128  0.0000819  2.8944823 22.2377062  0.0000007
  0.0000006]

[ 0.0000254  0.0000076  0.0000489  1.0452399 40.852684   0.0000007
  0.0000007]

[ 0.0000218  0.0000118  0.0000791  0.        17.0576003  0.0000007
  0.0000006]

[ 0.0000213  0.0000104  0.0000815  0.        26.4317844  0.0000007
  0.0000006]

[ 0.0000169  0.000008   0.0000732  3.3711166 63.3951127  0.0000007
  0.0000006]

[ 0.0000266  0.0000082  0.000095   0.        22.1743263  0.0000007
  0.0000006]

[ 0.0000181  0.0000028  0.0000828  0.        43.6669396  0.0000007
  0.0000006]

[0.0000296 0.0000098 0.0000831 0.        4.4056284 0.0000006 0.0000006]

[ 0.0000262  0.0000069  0.0000613 17.1730489 25.147664   0.0000006
  0.0000006]

[ 0.0000234  0.000004   0.000072   0

In [63]:
# feature, target 값 정의
feature = df[['lnnz','rnnz']]
feature = feature.astype({'lnnz': 'int','rnnz': 'int'})
target = df['총메모리합']

epoch = 100
mape = []
r2 = []

for i in range(epoch):
    
    # 훈련 데이터와 테스트 데이터 분리
    X_train, X_test, Y_train, Y_test = train_test_split(feature, target, test_size = 0.1)

    # NNLS(Non-Negative Least Squares)회귀모델 정의 및 훈련데이터로 학습
    # (모든 가중치가 0 또는 양수값을 가진다.)
    np.set_printoptions(precision=8, suppress=True)
    nnls_coef, nnls_intercept =nnls(X_train.to_numpy(), Y_train.to_numpy(), maxiter=1000)

    # NNLS를 통해 나온 각 weight 추출
    C1=np.array(pd.DataFrame(nnls_coef).iloc[0])
    C2=np.array(pd.DataFrame(nnls_coef).iloc[1])


    # 테스트 데이터 예측값 도출
    nnls_Y_pred = C1*X_test['lnnz']+C2*X_test['rnnz']

    # 모델의 weight(가중치)와 bias(절편) 출력
    #print("nnls_coef : ",nnls_coef)
    #print("\nnnls_intercept : ",nnls_intercept)

    # nnls metric
    #print("\nMAPE : ", mean_absolute_percentage_error(Y_test,nnls_Y_pred))
    #print("R^2 : ", r2_score(Y_test, nnls_Y_pred))

    print(str(nnls_coef) + "\n")    
    
    mape.append(mean_absolute_percentage_error(Y_test,nnls_Y_pred))
    r2.append(r2_score(Y_test, nnls_Y_pred))

print("MAPE : ", sum(mape)/len(mape) )
print("R^2 : ", sum(r2)/len(r2) )

[0.00000071 0.00000072]

[0.00000072 0.00000071]

[0.00000071 0.00000071]

[0.00000073 0.00000068]

[0.00000072 0.00000072]

[0.00000072 0.0000007 ]

[0.00000072 0.00000071]

[0.00000072 0.00000071]

[0.00000072 0.0000007 ]

[0.00000072 0.00000071]

[0.00000072 0.00000072]

[0.00000071 0.00000071]

[0.00000072 0.0000007 ]

[0.00000072 0.0000007 ]

[0.00000072 0.00000071]

[0.00000071 0.00000072]

[0.00000071 0.00000071]

[0.00000071 0.00000072]

[0.00000071 0.00000071]

[0.00000074 0.0000007 ]

[0.00000073 0.0000007 ]

[0.00000072 0.0000007 ]

[0.00000071 0.00000071]

[0.00000071 0.00000071]

[0.00000072 0.00000069]

[0.00000073 0.00000069]

[0.00000073 0.0000007 ]

[0.00000074 0.0000007 ]

[0.00000072 0.00000071]

[0.00000071 0.00000072]

[0.00000072 0.00000072]

[0.00000073 0.00000071]

[0.00000071 0.00000071]

[0.00000073 0.00000071]

[0.00000072 0.00000071]

[0.00000072 0.00000071]

[0.00000071 0.00000072]

[0.00000072 0.00000071]

[0.00000073 0.00000071]

[0.00000072 0.0000007 ]

